In [1]:
from djitellopy import Tello
import cv2
import numpy as np
import time

#########
# DETECTION
#########
# Load COCO class names
with open('COCO/object_detection_classes_coco.txt', 'r') as f:
    class_names = f.read().split('\n')

# Generate random colors for each class
COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))

# Load the pretrained SSD MobileNet model
model = cv2.dnn.readNet(model='COCO/frozen_inference_graph.pb',
                        config='COCO/ssd_mobilenet_v2_coco_2018_03_29.pbtxt.txt',
                        framework='TensorFlow')

# Create and connect to Tello
tello = Tello()
print("Connecting to Tello...")
tello.connect()
print(f"Battery level: {tello.get_battery()}%")

# Start video stream
tello.streamon()

####
# Initialize Camera
#####
print("Press 't' to takeoff, 'q' to land and quit.")

# Drone state
drone_in_air = False

while True:
    # Get video frame from Tello
    frame = tello.get_frame_read().frame
    frame = cv2.resize(frame, (720, 480))

    image_height, image_width, _ = frame.shape

    # Create blob from image
    blob = cv2.dnn.blobFromImage(image=frame, size=(300, 300), mean=(104, 117, 123), swapRB=True)

    # Set input and run forward pass
    model.setInput(blob)
    detections = model.forward()

    # Loop through detections
    for detection in detections[0, 0, :, :]:
        confidence = detection[2]
        class_id = int(detection[1])

        if confidence > 0.4 and class_id == 1:  # Only detect 'person'
            class_name = class_names[class_id - 1]
            color = COLORS[class_id]

            box_x = int(detection[3] * image_width)
            box_y = int(detection[4] * image_height)
            box_width = int(detection[5] * image_width)
            box_height = int(detection[6] * image_height)

            # Draw rectangle and label
            cv2.rectangle(frame, (box_x, box_y), (box_width, box_height), color, 2)
            label = f"{class_name} {confidence:.2f}"
            cv2.putText(frame, label, (box_x, box_y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # Show video feed
    cv2.imshow("Tello Detection Feed", frame)

    # Check for user input
    key = cv2.waitKey(1) & 0xFF

    # Press 't' to take off if drone is on the ground
    if key == ord('t') and not drone_in_air:
        tello.takeoff()
        drone_in_air = True
        print("Drone is now airborne and hovering.")

    # Press 'q' to land and quit
    if key == ord('q'):
        if drone_in_air:
            tello.land()
            print("Drone has landed.")
        break

# Land drone and stop stream
tello.streamoff()
cv2.destroyAllWindows()


[INFO] tello.py - 129 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 438 - Send command: 'command'
[INFO] tello.py - 462 - Response command: 'ok'
[INFO] tello.py - 438 - Send command: 'streamon'


Connecting to Tello...
Battery level: 67%


[INFO] tello.py - 462 - Response streamon: 'ok'


Press 't' to takeoff, 'q' to land and quit.


2025-04-19 16:45:05.253 Python[17105:4119753] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.
[INFO] tello.py - 438 - Send command: 'takeoff'
[INFO] tello.py - 462 - Response takeoff: 'ok'


Drone is now airborne and hovering.


[INFO] tello.py - 438 - Send command: 'land'
[INFO] tello.py - 462 - Response land: 'ok'
[INFO] tello.py - 438 - Send command: 'streamoff'
[INFO] tello.py - 462 - Response streamoff: 'ok'


Drone has landed.
